# Load Data

In [1]:
# Load in the python script containing the same code as the load the data notebook
%run loadData.py
# now we can access train, dev, and test
# along with trainSents, devSents testSents

# Demo Data

In [2]:
trainSents[0][2]

u'Phonograph records are generally described by their diameter in inches (12", 10", 7"), the rotational speed in rpm at which they are played (16 2\u20443, 33 1\u20443, 45, 78), and their time capacity resulting from a combination of those parameters (LP \u2013 long playing 33 1\u20443 rpm, SP \u2013 78 rpm single, EP \u2013 12-inch single or extended play, 33 or 45 rpm); their reproductive quality or level of fidelity (high-fidelity, orthophonic, full-range, etc.), and the number of audio channels provided (mono, stereo, quad, etc.).'

In [3]:
train[0][0]

{u'answer': u'long playing',
 u'answer_sentence': 2,
 u'question': u'What does LP stand for when it comes to time capacity?'}

In [4]:
documents = trainSents[0]
questions = train[0]

# Useful Imports

In [5]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

# Shared Workflow Thoughts (dealing with .ipynb notebooks)

Think with each feature we do below, create generalized functions that can be easily composed with easy names, split by type.

Then create/use small demo template below using the locked document/questions above to get intuition, check sanity, iterate quickly, to help keep us all on the same page.

This way we keep everything well contained/documented/explainable, will help with report writing.

Then in separate notebook we do statistical valid/testing for error exploration/analysis, using generalized functions above - easily changeable/copyable.

Finally put it all in a python file that will do full run. Write TODOs to illustrate next steps/improvements, that way can stay on top/track/improve upon easily.

# Sentence Retreival

The first part of your basic QA system will use a bag-of-words (BOW) vector space model to identify the sentence in the Wikipedia article which is most likely to contain the answer to a question, using standard information retrieval techniques. Here the "query" is the question, the "documents" are actually sentences, and each Wikipedia article should be viewed as separate "document collection". You should apply various preprocessing steps appropriate to this situation, including term weighting; if you are at all uncertain about what choices to make, you should evaluate them using the dev data, and use the results to justify your choice in your report.

##### TODO

* Improving tuning of preprocessing/lemmatize functions for use QA case

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [7]:
# Tuning functions

import nltk
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

# Follow lemmatize function from guide notebook: WSTA_N1B_preprocessing.ipynb
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma

In [8]:
# Core functions

def vectorize_documents(text_documents):

    vectorizer = TfidfVectorizer(stop_words='english')
    vector_documents = vectorizer.fit_transform(text_documents)
    
    return [vector_documents, vectorizer]

def vectorize_query(vectorizer, text_query):
    return vectorizer.transform([text_query])

def process_neighbours(vector_documents):
    
    neighbours = NearestNeighbors(1, algorithm="brute", metric="cosine")
    neighbours.fit(vector_documents)
    
    return neighbours

def closest_document(neighbours, vector_query):

    result = neighbours.kneighbors(vector_query, 1, return_distance=True)

    result_index = result[1][0][0]
    result_similarity = result[0][0][0]
    
    return [result_similarity, result_index]

In [9]:
# Demonstration function

def demo_process_set(questions, documents):
    
    vector_documents, vectorizer = vectorize_documents(documents)
    analyze = vectorizer.build_analyzer()
    neighbours = process_neighbours(vector_documents)

    print "=" * 20
    print "Vector documents shape: {0}".format(vector_documents.shape)
    print "Actual documents length: {0}".format(len(documents))
    print "=" * 20, "\n"
    
    for question in questions[10:10+3]:
        
        text_query = question["question"]

        print "Text query:\n\n\t{0}\n".format(text_query)

        vector_query = vectorize_query(vectorizer, text_query)

        print "Vector query shape:\n\n\t{0}".format(vector_query.shape)

        result_similarity, result_index  = closest_document(neighbours, vector_query)
        
        print

        print "Result:\n\n\tSimilarity ({0}), Index ({1})\n".format(result_similarity, result_index)

        print

        print "Query (text):\n\n\t{0}\n".format(text_query)
        print "Document (text):\n\n\t{0}".format(documents[result_index].encode("utf-8"))

        print

        print "Query (vector text):\n"
        pp.pprint(analyze(text_query))
        print
        
        print "Document (vector text): \n\n"
        pp.pprint(analyze(documents[result_index]))
        
        print "\n", "=" * 20, "\n"

In [10]:
demo_process_set(questions, documents)

Vector documents shape: (463, 2425)
Actual documents length: 463

Text query:

	What was the original intent of the phonautograph?

Vector query shape:

	(1, 2425)

Result:

	Similarity (0.72338690096), Index (9)


Query (text):

	What was the original intent of the phonautograph?

Document (text):

	The phonautograph, patented by Léon Scott in 1857, used a vibrating diaphragm and stylus to graphically record sound waves as tracings on sheets of paper, purely for visual analysis and without any intent of playing them back.

Query (vector text):

[u'original', u'intent', u'phonautograph']

Document (vector text): 


[   u'phonautograph',
    u'patented',
    u'l\xe9on',
    u'scott',
    u'1857',
    u'used',
    u'vibrating',
    u'diaphragm',
    u'stylus',
    u'graphically',
    u'record',
    u'sound',
    u'waves',
    u'tracings',
    u'sheets',
    u'paper',
    u'purely',
    u'visual',
    u'analysis',
    u'intent',
    u'playing']


Text query:

	In what years where phonauto

# Entity Extraction

The second main part of your basic QA system is an NER system. In this initial system you should have at least four
answer types: PERSON, LOCATION, NUMBER, and OTHER. You should run the Stanford NER system over your
sentences to extract people and location entities (Hint: make use of the "tag_sents" method in the NLTK interface to do
this efficiently for multiple sentences in a single call, otherwise this will be very slow; you may also want to cache the
entity information during development of your system, rather than calling Stanford NER for each run). Note that
contiguous words tagged as the same type should be considered part of the same entity. ORGANIZATION entities
extracted by the NER system should be considered OTHER. You should also extract and treat as OTHER any other
non-sentence initial sequence of capitalized words not tagged by Stanford NER. Finally, you should label all numbers as
NUMBER. In this process, you might notice errors related to your preprocessing (e.g. tokenization), errors which can be
easily corrected should be addressed at this stage.

##### TODO

* Finish NER Tagging 
* Test run stanford function on another machine (+ record times)

In [11]:
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize

# The required jar files : https://nlp.stanford.edu/software/CRF-NER.shtml#Download
# It's 171mb so I've added to the gitignore
# If you download it, and rename the folder name "stanford" in the main directory
classifier = './stanford/classifiers/english.all.3class.distsim.crf.ser.gz'
jar = './stanford/stanford-ner.jar'

sTagger = StanfordNERTagger(classifier,jar)

In [12]:
#Quick test of Tagger
text = "There were three geese, Derek, Joshi, and Alex, who lived in Melbourne, and worked at Microsoft"
tokenizedText = word_tokenize(text)
classifiedText = sTagger.tag_sents([tokenizedText])

# Warning, it takes a little while ~ 5 seconds on my comp
print(classifiedText)

[[(u'There', u'O'), (u'were', u'O'), (u'three', u'O'), (u'geese', u'O'), (u',', u'O'), (u'Derek', u'PERSON'), (u',', u'O'), (u'Joshi', u'PERSON'), (u',', u'O'), (u'and', u'O'), (u'Alex', u'PERSON'), (u',', u'O'), (u'who', u'O'), (u'lived', u'O'), (u'in', u'O'), (u'Melbourne', u'LOCATION'), (u',', u'O'), (u'and', u'O'), (u'worked', u'O'), (u'at', u'O'), (u'Microsoft', u'ORGANIZATION')]]


In [13]:
import pickle # Useful for read / write of list file
import os #Needed to check if file exists

In [14]:
# Lets store the stanford tagger output in a file
# This function returns the tagging output of stanford for each dataset
# with datasetName - 'train', 'dev', test' 

def getStanfordTagging(datasetName):
    fnameTrain = './preCompTags/stanfordTaggedTrain.txt'
    fnameDev = './preCompTags/stanfordTaggedDev.txt'
    fnameTest = './preCompTags/stanfordTaggedTest.txt'
    
    theFilePath = ''
    theSents = []
    if (datasetName == 'train'):
        theFilePath = fnameTrain
        theSents = trainSents
    elif (datasetName == 'dev'):
        theFilePath = fnameDev
        theSents = devSents
    elif (datasetName == 'test'):
        theFilePath = fnameTest
        theSents = testSents
    else :
        raise ValueError('Incorrect datasetName: ' + datasetName + ', choose from - "train", "dev", "test" ') 
    if (os.path.exists(theFilePath)):
        with open(theFilePath, "rb") as fp:
            stanfordTags = pickle.load(fp)
            return stanfordTags
    
    else :
        #Need to create taggings!
        taggedSentsList = []
        for sents in theSents:
            tokenisedSents = [word_tokenize(sent) for sent in sents]
            classifiedSents = sTagger.tag_sents(tokenisedSents)
            taggedSentsList.append(classifiedSents)
        #And save them
        with open(theFilePath, "wb") as fp: 
            pickle.dump(taggedSentsList, fp)
        return taggedSentsList
    

In [15]:
taggedTrain = getStanfordTagging('train')
taggedDev = getStanfordTagging('dev')
taggedTest = getStanfordTagging('test')

In [16]:
import re

# Given a stanford tagged list, refines the list by:
# Grouping all contiguous words with the same tag
# Relabels Organisations as Other
# Labels Number

def refineWordTags(taggedWordList):
    newWordTags = []
    prevWord = ''
    prevTag = taggedWordList[0][1] # Ie the first tag
    for (word, tag) in taggedWordList:
        if (tag == 'ORGANIZATION'):
            tag = 'O'
        if (tag == 'O'):
            #Might be a number
            if isNumber(word):
                tag = 'NUMBER'
        if (tag == prevTag):
            prevWord += ' ' + word
        else :
            newWordTags.append((prevWord, prevTag))
            prevWord = word
            prevTag = tag
    # Need to add the final ones
    newWordTags.append((prevWord, prevTag))
    return newWordTags
        

# Returns true if the word represents a number
def isNumber(word):
    pattern = ".?(\d)+((,|.)(\d)+)*"
    if re.match(pattern,word) :
        return True
    return False


In [17]:
# Small test:
print (taggedTrain[1][0])
print
print refineWordTags(taggedTrain[1][0])

[(u'Raleigh', u'O'), (u'(', u'O'), (u'\u02c8r\u0251\u02d0li', u'O'), (u';', u'O'), (u'RAH-lee', u'O'), (u')', u'O'), (u'is', u'O'), (u'the', u'O'), (u'capital', u'O'), (u'of', u'O'), (u'the', u'O'), (u'state', u'O'), (u'of', u'O'), (u'North', u'LOCATION'), (u'Carolina', u'LOCATION'), (u'as', u'O'), (u'well', u'O'), (u'as', u'O'), (u'the', u'O'), (u'seat', u'O'), (u'of', u'O'), (u'Wake', u'LOCATION'), (u'County', u'LOCATION'), (u'in', u'O'), (u'the', u'O'), (u'United', u'LOCATION'), (u'States', u'LOCATION'), (u'.', u'O')]

[(u' Raleigh ( \u02c8r\u0251\u02d0li ; RAH-lee ) is the capital of the state of', u'O'), (u'North Carolina', u'LOCATION'), (u'as well as the seat of', u'O'), (u'Wake County', u'LOCATION'), (u'in the', u'O'), (u'United States', u'LOCATION'), (u'.', u'O')]


In [18]:
#Quick regex test:
print isNumber('.25')
print isNumber('123.123')
print isNumber('163.2342.234  ')
print isNumber('123,123,123.2  ')
print isNumber('joshi')
print isNumber('$123')
print isNumber('four') # I guess ideally this should be true
print isNumber('h123') # Should this be a number?

True
True
True
True
False
True
False
True


# Answer Ranking

After you have extracted a set of entities from your sentence, you will rank them to choose the best answer. The ranking
should be based on three factors. First, answers whose content words all appear in the question should be ranked
lowest. Second, answers which match the question type should be ranked higher than those that don't; for this, you
should build a simple rule-based question type classifier based on key words (e.g. questions which contain "who" are
people). Third, among entities of the same type, the prefered entity should be the one which is closer in the sentence to a
closed-class word from the question.

##### TODO

* TODO

In [19]:
vectorize_query("What is your name there son?")

TypeError: vectorize_query() takes exactly 2 arguments (1 given)

In [ ]:
# Part A gives us a most likely sentence
# Part B splits into entities


# Given a question, returns a tag for the answer form
# From PERSON, LOCATION, NUMBER, OTHER 
# Assuming question is lowercased
def getQuestionType(question):
    if 'who' in question:
        return "PERSON"
    if 'where' in question:
        return "LOCATION"
    return "OTHER"

## Evaluation 
I've added the evaluation here to save exporting the above functions


## Sentence Retrival


In [47]:
# Where numToEval is max 360 for train
def evaluateSentenceRetrival(questionsList,documentsList, numToEval):
    numCorrect = 0
    numWrong = 0
    for i in range (0, numToEval):
        documents = documentsList[i]
        questions = questionsList[i]
        
        vector_documents, vectorizer = vectorize_documents(documents)
        analyze = vectorizer.build_analyzer()
        neighbours = process_neighbours(vector_documents)
        
        for j in range (0, len(questions)):
            text_query = questions[j]["question"]
            vector_query = vectorize_query(vectorizer, text_query)
            result_similarity, result_index  = closest_document(neighbours, vector_query)
            if result_index == int(questions[j]["answer_sentence"]):
                numCorrect += 1
            else :
                numWrong += 1
    return (numCorrect, numWrong)

In [48]:
evaluateSentenceRetrival(train, trainSents,10)

(905, 756)